In [43]:
# 사용할 툴 가져오기
from bs4 import BeautifulSoup as bs # 크롤링을 위한 툴(라이브러리)
import requests # 파싱을 위한 툴(라이브러리)
import pymongo # 데이터베이스 연동을 위한 툴(라이브러리)
import re

# 데이터베이스와 컬렉션 생성
conn = pymongo.MongoClient()
actor_db = conn.cine21
actor_col = actor_db.actor_collection

In [25]:
url = 'http://www.cine21.com/rank/person/content' # 해당 페이지의 content를 가져올 것을 명시
post_data = dict() # post 형식으로 받아오는 data를 dictionary 형태로 가져올 것임
post_data['section'] = 'actor'
post_data['period_start'] = '2022-01' # 집계기간 1년 기준
post_data['gender'] = 'all' 
post_data['page'] = 1

res = requests.post(url, data=post_data) # post 메서드이므로 data를 입력해줘야된다

In [26]:
soup = bs(res.content, 'html.parser') 

# 1. 배우 이름 가져오기
# people_li 클래스 명을 가진 li 태그 하위 태그중 name 클래스명을 가진 div 태그 불러오기
actors = soup.select('li.people_li div.name') 
for actor in actors:
    str_ = actor.text
    for i in range(len(str_)):
        if str_[i] == '(': break
    print(str_[:i])

현빈
유해진
박해일
임윤아
박지환
손석구
최귀화


In [54]:
# 2. 배우 상세 정보 가져오기
actor_list = []
for actor in actors:
    # 2-1. 배우 상세정보 페이지 이동
    a_url = 'http://www.cine21.com' + actor.select_one('a').attrs['href']
    
    # 2-2. 해당 페이지 html 코드 가져오기
    a_res = requests.get(a_url)
    a_soup = bs(a_res.content, 'html.parser')
    detail_info = a_soup.select('ul.default_info li')
        
    # 2-3. mongodb 형식으로 만들기 (field와 value로 나누기)
    # span 태그의 text가 필드명으로 적합하다고 판단
    actor_dict = dict()
    for detail in detail_info:
        a_field = (detail.select_one('span.tit').text)
        a_value = (detail.text[len(detail.select_one('span.tit').text):].replace('\n', ''))
        actor_dict[a_field] = a_value
    actor_list.append(actor_dict)

In [2]:
from bs4 import BeautifulSoup as bs
import requests
import pymongo
from datetime import date, timedelta, datetime
from dateutil.relativedelta import relativedelta

today = date.today() - relativedelta(months=6)
today = today.isoformat()[:7]

conn = pymongo.MongoClient()
actor_db = conn.cine21
actor_col = actor_db.actor_col_6M

actor_col.delete_many({})

url = 'http://www.cine21.com/rank/person/content' # 해당 페이지의 content를 가져올 것을 명시
post_data = dict() # post 형식으로 받아오는 data를 dictionary 형태로 가져올 것임
post_data['section'] = 'actor'
post_data['period_start'] = today
post_data['gender'] = 'all' 
actor_list = []
for p in range(1, 21):
    post_data['page'] = p

    res = requests.post(url, data=post_data)
    soup = bs(res.content, 'html.parser') 

    # 데이터 전처리
    actors = soup.select('li.people_li div.name')
    hits = soup.select('ul.num_info li strong')
    movies = soup.select('ul.mov_list')
    
    for ind, actor in enumerate(actors):
        actor_info={}

        a_name = actor.select_one('a').text
        for i in range(len(a_name)):
            if a_name[i] == '(': break
        a_name = a_name[:i]

        hit = hits[ind].text.replace(',','')

        movie_titles = movies[ind].select('li a span')
        movie_title_list = []
        for movie_title in movie_titles:
            movie_title_list.append(movie_title.text)
        actor_info['배우이름'] = a_name
        actor_info['흥행지수'] = int(hit)
        actor_info['출연영화'] = movie_title_list

        a_url = 'http://www.cine21.com' + actor.select_one('a').attrs['href']

        a_res = requests.get(a_url)
        a_soup = bs(a_res.content, 'html.parser')
        detail_info = a_soup.select('ul.default_info li')

        actor_info_detail = {}
        for detail in detail_info:
            a_field = (detail.select_one('span.tit').text)
            a_value = (detail.text[len(detail.select_one('span.tit').text):].replace('\n', ''))
            actor_info_detail[a_field] = a_value
        actor_info['상세정보'] = actor_info_detail
        actor_list.append(actor_info)

print('before insert : ' + str(actor_col.count_documents({})))
actor_col.insert_many(actor_list)
print('after insert : ' + str(actor_col.count_documents({})))

before insert : 0
after insert : 140


In [112]:
actors = actor_col.find()
for a in actors:
    print(a)

{'_id': ObjectId('63d36715bc00af76e5d8abb1'), '배우이름': '현빈', '흥행지수': 65154, '출연영화': ['공조2: 인터내셔날', '교섭', '만추', '공조'], '상세정보': {'원어명': '玄彬', '다른 이름': '김태평', '직업': '배우', '생년월일': '1982-09-25', '성별': '남', '홈페이지': 'http://www.vastenm.com/', '신장/체중': '184cm, 74kg', '학교': '중앙대학교 연극영화 졸업, 중앙대학교 예술 대학원 공연영상학과 재학 중', '취미': '수영, 농구, 야구', '특기': '스노우보드, 사진', '소속사': 'VAST엔터테인먼트'}}
{'_id': ObjectId('63d36715bc00af76e5d8abb2'), '배우이름': '유해진', '흥행지수': 58978, '출연영화': ['공조2: 인터내셔날', '올빼미', '말모이', '공조', '택시운전사'], '상세정보': {'다른 이름': 'YOO HAI JIN', '직업': '배우', '생년월일': '1970-01-04', '성별': '남', '신장/체중': '174cm, 65kg', '학교': '서울예술대학 연극과', '소속사': '심엔터테인먼트'}}
{'_id': ObjectId('63d36715bc00af76e5d8abb3'), '배우이름': '박해일', '흥행지수': 38010, '출연영화': ['헤어질 결심', '한산: 용의 출현', '한산 리덕스', '고령화가족', '살인의 추억', '후아유'], '상세정보': {'다른 이름': 'Park Hae-IL', '직업': '배우', '생년월일': '1977-01-26', '성별': '남', '신장/체중': '177cm, 65kg', '학교': '남서울대학교 영어학 (중퇴)', '취미': '영화감상, 음악감상, 컴퓨터게임', '특기': '노래, 기타연주, 탁구, 야구', '소속사': '휴메인엔터테인먼트'}}
{'_id': ObjectI

In [111]:
actors = actor_col.find({'상세정보.직업' : '가수'}, {'_id' : 0, '배우이름' : 1, '흥행지수':1, 'ranking':1}).sort('흥행지수', pymongo.DESCENDING)
for ind, a in enumerate(actors):
    print(str(ind+1) + ' :', end=' ')
    print(a)

1 : {'배우이름': '임윤아', '흥행지수': 33152}
2 : {'배우이름': '옥택연', '흥행지수': 9628}
3 : {'배우이름': '나나', '흥행지수': 3399}
4 : {'배우이름': '옹성우', '흥행지수': 2723}
5 : {'배우이름': '서인국', '흥행지수': 1806}
6 : {'배우이름': '차은우', '흥행지수': 1442}
7 : {'배우이름': '윤두준', '흥행지수': 972}
8 : {'배우이름': '박서함', '흥행지수': 849}


In [94]:
actors = actor_col.find(
    {'$or' : [
        {'출연영화' : '공조2: 인터내셔날'}, 
        {'출연영화' : '공조'}
    ]}, 
    {'_id' : 1, '배우이름' : 1, '출연영화' : 1} )
for a in actors:
    print(a)

{'_id': ObjectId('63d36715bc00af76e5d8abb1'), '배우이름': '현빈', '출연영화': ['공조2: 인터내셔날', '교섭', '만추', '공조']}
{'_id': ObjectId('63d36715bc00af76e5d8abb2'), '배우이름': '유해진', '출연영화': ['공조2: 인터내셔날', '올빼미', '말모이', '공조', '택시운전사']}
{'_id': ObjectId('63d36715bc00af76e5d8abb4'), '배우이름': '임윤아', '출연영화': ['공조2: 인터내셔날', '엑시트', '기적', '공조']}
{'_id': ObjectId('63d36715bc00af76e5d8abb9'), '배우이름': '진선규', '출연영화': ['공조2: 인터내셔날', '태일이', '불한당: 나쁜 놈들의 세상', '사바하']}
{'_id': ObjectId('63d36715bc00af76e5d8abd1'), '배우이름': '이이경', '출연영화': ['육사오(6/45)', '심야카페: 미씽 허니', '공조']}
{'_id': ObjectId('63d36715bc00af76e5d8ac37'), '배우이름': '장영남', '출연영화': ['늑대사냥', '늑대소년', '국제시장', '미쓰백', '증인', '공조']}


In [84]:
actors = actor_col.find(
    {'출연영화' : {'$all' : ['공조2: 인터내셔날', '공조'] } }, 
    {'_id' : 0, '배우이름' : 1, '출연영화' : 1} )

for a in actors:
    print(a)

{'배우이름': '현빈', '출연영화': ['공조2: 인터내셔날', '교섭', '만추', '공조']}
{'배우이름': '유해진', '출연영화': ['공조2: 인터내셔날', '올빼미', '말모이', '공조', '택시운전사']}
{'배우이름': '임윤아', '출연영화': ['공조2: 인터내셔날', '엑시트', '기적', '공조']}


In [86]:
actors = actor_col.find(
    {'출연영화.0' : '공조2: 인터내셔날'}, 
    {'_id' : 0, '배우이름' : 1, '출연영화' : 1} )

for a in actors:
    print(a)

{'배우이름': '현빈', '출연영화': ['공조2: 인터내셔날', '교섭', '만추', '공조']}
{'배우이름': '유해진', '출연영화': ['공조2: 인터내셔날', '올빼미', '말모이', '공조', '택시운전사']}
{'배우이름': '임윤아', '출연영화': ['공조2: 인터내셔날', '엑시트', '기적', '공조']}
{'배우이름': '진선규', '출연영화': ['공조2: 인터내셔날', '태일이', '불한당: 나쁜 놈들의 세상', '사바하']}


In [7]:
actors = actor_col.find(
    {'출연영화' : {'$size' : 5}}, 
    {'_id' : 0, '배우이름' : 1, '출연영화' : 1} )

for a in actors:
    print(a)

{'배우이름': '유해진', '출연영화': ['공조2: 인터내셔날', '올빼미', '말모이', '공조', '택시운전사']}
{'배우이름': '정우성', '출연영화': ['헌트', '더 킹', '강철비2: 정상회담', '증인', '아수라']}
{'배우이름': '나문희', '출연영화': ['영웅', '룸 쉐어링', '오! 문희', '감쪽같은 그녀', '수상한 그녀']}
{'배우이름': '김향기', '출연영화': ['한산: 용의 출현', '한산 리덕스', '영주', '늑대소년', '증인']}
{'배우이름': '이병헌', '출연영화': ['비상선언', '백두산', '공동경비구역 JSA', '그것만이 내 세상', '번지점프를 하다']}
{'배우이름': '염정아', '출연영화': ['외계+인 1부', '인생은 아름다워', '카트', '장화, 홍련', '강철비2: 정상회담']}
{'배우이름': '이성민', '출연영화': ['리멤버', '군도 : 민란의 시대', '기적', '변호인', '밀양']}
{'배우이름': '김의성', '출연영화': ['외계+인 1부', '부산행', '더 킹', '암살', '남영동1985 ']}
{'배우이름': '김서영', '출연영화': ['뽀로로 극장판 드래곤캐슬 대모험', '극장판 윌벤져스: 수상한 캠핑대소동', '엄마의 땅: 그리샤와 숲의 주인', '극장판 엄마 까투리: 도시로 간 까투리 가족', '날씨의 아이']}
{'배우이름': '박성웅', '출연영화': ['젠틀맨', '대무가', '무뢰한', '오케이 마담', '신세계']}


In [89]:
actors = actor_col.find({'흥행지수' : {'$elemMatch' : {'$gte' : 10000, '$lte' : 50000}}})
for a in actors:
    print(a)

In [105]:
actor_col.create_index('배우이름')

'배우이름_1'

In [117]:
actor_col.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)], 'ns': 'cine21.actor_col_6M'}}

In [39]:
actor_col.drop_indexes()

In [107]:
actor_col.drop_index([('배우이름', 1)])

In [113]:
actor_col.create_index('배우이름')
actor_col.create_index('흥행지수')
actor_col.create_index('출연영화')
actor_col.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)], 'ns': 'cine21.actor_col_6M'},
 '배우이름_1': {'v': 2, 'key': [('배우이름', 1)], 'ns': 'cine21.actor_col_6M'},
 '흥행지수_1': {'v': 2, 'key': [('흥행지수', 1)], 'ns': 'cine21.actor_col_6M'},
 '출연영화_1': {'v': 2, 'key': [('출연영화', 1)], 'ns': 'cine21.actor_col_6M'}}

In [114]:
actor_col.drop_index([('배우이름', 1)])
actor_col.create_index([('배우이름', pymongo.DESCENDING)])

'배우이름_-1'

In [124]:
actor_col.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)], 'ns': 'cine21.actor_col_6M'},
 '배우이름_text': {'v': 2,
  'key': [('_fts', 'text'), ('_ftsx', 1)],
  'ns': 'cine21.actor_col_6M',
  'weights': SON([('배우이름', 1)]),
  'default_language': 'english',
  'language_override': 'language',
  'textIndexVersion': 3}}

In [120]:
actor_col.create_index([('배우이름', 1), ('흥행지수', 1), ('출연영화', -1)])
actor_col.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)], 'ns': 'cine21.actor_col_6M'},
 '배우이름_1_흥행지수_1_출연영화_-1': {'v': 2,
  'key': [('배우이름', 1), ('흥행지수', 1), ('출연영화', -1)],
  'ns': 'cine21.actor_col_6M'}}

In [41]:
actor_col.create_index([('상세정보.직업', 'text')])

'상세정보.직업_text'

In [42]:
actor_col.index_information()

{'_id_': {'v': 2, 'key': [('_id', 1)], 'ns': 'cine21.actor_col_6M'},
 '상세정보.직업_text': {'v': 2,
  'key': [('_fts', 'text'), ('_ftsx', 1)],
  'ns': 'cine21.actor_col_6M',
  'weights': SON([('상세정보.직업', 1)]),
  'default_language': 'english',
  'language_override': 'language',
  'textIndexVersion': 3}}

In [48]:
actor = actor_col.find({'$text' : {'$search' : '가수 배우'}}, {'_id' : 0, '상세정보.학교' : 1, '배우이름' : 1})
for a in actor:
    print(a)

{'배우이름': '박서함', '상세정보': {}}
{'배우이름': '윤두준', '상세정보': {}}
{'배우이름': '차은우', '상세정보': {}}
{'배우이름': '서인국', '상세정보': {'학교': '경희사이버대학교 정보통신학'}}
{'배우이름': '옹성우', '상세정보': {}}
{'배우이름': '나나', '상세정보': {}}
{'배우이름': '옥택연', '상세정보': {'학교': '단국대학교 경영학'}}
{'배우이름': '임윤아', '상세정보': {}}
{'배우이름': '조민수', '상세정보': {}}
{'배우이름': '배인혁', '상세정보': {}}
{'배우이름': '남문철', '상세정보': {}}
{'배우이름': '장영남', '상세정보': {}}
{'배우이름': '정재광', '상세정보': {}}
{'배우이름': '이민기', '상세정보': {'학교': '건국대학교 영화예술학 (재학)'}}
{'배우이름': '하준', '상세정보': {}}
{'배우이름': '고창석', '상세정보': {}}
{'배우이름': '성유빈', '상세정보': {}}
{'배우이름': '김동희', '상세정보': {}}
{'배우이름': '허동원', '상세정보': {}}
{'배우이름': '정소민', '상세정보': {'학교': '한국예술종합학교 연극원 연기과'}}
{'배우이름': '손석구', '상세정보': {}}
{'배우이름': '진구', '상세정보': {'학교': '삼육의명대학 광고정보'}}
{'배우이름': '박소이', '상세정보': {}}
{'배우이름': '박호산', '상세정보': {}}
{'배우이름': '최귀화', '상세정보': {}}
{'배우이름': '성동일', '상세정보': {'학교': '유한대학 기계설계학'}}
{'배우이름': '서은수', '상세정보': {}}
{'배우이름': '나인우', '상세정보': {'학교': '단국대학교 공연영화학부'}}
{'배우이름': '박해수', '상세정보': {}}
{'배우이름': '김민호', '상세정보': {'학교': '경희대학교 연극영화학'}}


In [51]:
actor = actor_col.find({'상세정보.학교': {'$regex' : '중앙대학교'}}, {'_id': 0, '배우이름':1,'상세정보.학교':1}).limit(10)
for a in actor:
    print(a)

{'배우이름': '현빈', '상세정보': {'학교': '중앙대학교 연극영화 졸업, 중앙대학교 예술 대학원 공연영상학과 재학 중'}}
{'배우이름': '손현주', '상세정보': {'학교': '중앙대학교 연극영화'}}
{'배우이름': '이정현', '상세정보': {'학교': '중앙대학교 연극영화'}}
{'배우이름': '박용우', '상세정보': {'학교': '중앙대학교 연극영화과'}}
{'배우이름': '김래원', '상세정보': {'학교': '중앙대학교 연극영화'}}
{'배우이름': '정경호', '상세정보': {'학교': '중앙대학교 연극학'}}
